# RAG from scratch

- Adapted from https://github.com/opendatahub-io/llama-stack-demos/
- Requires ollama template 

# Variables

In [18]:
base_url="http://localhost:8321"
chunk_size_in_tokens=512


In [1]:
import rich
from llama_stack_client import LlamaStackClient, RAGDocument


In [13]:
client = LlamaStackClient(
    base_url=base_url,
)

In [14]:
client.models.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/models "HTTP/1.1 200 OK"


[Model(identifier='llama3.2:3b-instruct-fp16', metadata={}, api_model_type='llm', provider_id='ollama', type='model', provider_resource_id='llama3.2:3b-instruct-fp16', model_type='llm'),
 Model(identifier='all-MiniLM-L6-v2', metadata={'embedding_dimension': 384.0}, api_model_type='embedding', provider_id='ollama', type='model', provider_resource_id='all-minilm:latest', model_type='embedding')]

In [15]:
from llama_stack_client import LlamaStackClient, RAGDocument

pdf_urls = [
    "https://arxiv.org/pdf/2304.08641.pdf",
    "https://www.cs.cmu.edu/~tom/mlbook/NBayesLogReg.pdf",
]

documents = [
    RAGDocument(
        document_id=f"pdf-{i}",
        content=url,  # still use `content` for the URL
        mime_type="application/pdf",
        metadata={"source": "arxiv" if "arxiv" in url else "cmu"},
    )
    for i, url in enumerate(pdf_urls)
]


In [24]:
urls = [
    ("https://www.openshift.guide/openshift-guide-screen.pdf", "application/pdf"),
]
documents = [
    RAGDocument(
        document_id=f"num-{i}",
        content=url,
        mime_type=url_type,
        metadata={},
    )
    for i, (url, url_type) in enumerate(urls)
]



In [25]:
documents

[{'document_id': 'num-0',
  'content': 'https://www.openshift.guide/openshift-guide-screen.pdf',
  'mime_type': 'application/pdf',
  'metadata': {}}]

In [27]:
vector_db_id = "demoInnovate-vectordb"
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model="all-MiniLM-L6-v2",
    embedding_dimension=384,
)

INFO:llama_stack_client._base_client:Retrying request to /v1/vector-dbs in 0.453451 seconds
INFO:llama_stack_client._base_client:Retrying request to /v1/vector-dbs in 0.781721 seconds


APIConnectionError: Connection error.

Ingesting documents into a vector database:

In [20]:
rich.print(documents)

[
    {
        'document_id': 'pdf-0',
        'content': 'https://arxiv.org/pdf/2304.08641.pdf',
        'mime_type': 'application/pdf',
        'metadata': {'source': 'arxiv'}
    },
    {
        'document_id': 'pdf-1',
        'content': 'https://www.cs.cmu.edu/~tom/mlbook/NBayesLogReg.pdf',
        'mime_type': 'application/pdf',
        'metadata': {'source': 'cmu'}
    }
]

In [22]:
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=512,
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/rag-tool/insert "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tool-runtime/rag-tool/insert in 0.426489 seconds
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/rag-tool/insert "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tool-runtime/rag-tool/insert in 0.761061 seconds
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/rag-tool/insert "HTTP/1.1 500 Internal Server Error"


InternalServerError: Error code: 500 - {'detail': 'Internal server error: An unexpected error occurred.'}

In [8]:
prompt = "What are the top 5 topics that were explained? Only list succinct bullet points."

In [9]:
# higher level tool provides packaged results, can span multiple dbs
tool_response = client.tool_runtime.rag_tool.query(
    content=prompt, vector_db_ids=[vector_db_id]
)
rich.print(tool_response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/tool-runtime/rag-tool/query "HTTP/1.1 200 OK"


QueryResult(
    metadata={'document_ids': ['num-3', 'num-3', 'num-3', 'num-3', 'num-3']},
    content=[
        TextContentItem(
            text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n',
            type='text'
        ),
        TextContentItem(
            text='Result 1\nContent: ,\n    and (b) the memory constraints of your hardware.\n\nThe preceding 
command will run a LoRA finetune with torchtune\'s factory settings, but we may want to experiment a bit.\nLet\'s 
take a closer look at some of the :code:`lora_finetune_distributed` config.\n\n.. code-block:: yaml\n\n  # Model 
Arguments\n  model:\n    _component_: lora_llama2_7b\n    lora_attn_modules: [\'q_proj\', \'v_proj\']\n    
lora_rank: 8\n    lora_alpha: 16\n  ...\n\nWe see that the default is to apply LoRA to Q and V projections with a 
rank of 8.\nSome experiments with LoRA have found that it can be beneficial to apply LoRA to all linear layers 
in\nthe self-attention, and to increase the rank to 16 or 32. Note that this is likely to increase our max 
memory,\nbut as long as we keep :code:`rank<<embed_dim`, the impact should be relatively minor.\n\nLet\'s run this 
experiment. We can also increase alpha (in general it is good practice to scale alpha and rank together).\n\n.. 
code-block:: bash\n\n    tune run --nnodes 1 --nproc_per_node 2 lora_finetune_distributed --config llama2/7B_lora 
\\\n    lora_attn_modules=[\'q_proj\',\'k_proj\',\'v_proj\',\'output_proj\'] \\\n    lora_rank=32 lora_alpha=64 
output_dir=./lora_experiment_1\n\nA comparison of the (smoothed) loss curves between this run and our baseline over
the first 500 steps can be seen below.\n\n.. image:: /_static/img/lora_experiment_loss_curves.png\n\n.. note::\n   
The above figure was generated with W&B. You can use torchtune\'s 
:class:`~torchtune.training.metric_logging.WandBLogger`\n    to generate similar loss curves, but you will need to 
install W&B and setup an account separately. For more details on\n    using W&B in torchtune, see our 
":ref:`wandb_logging`" recipe.\n\n.. _lora_tutorial_memory_tradeoff_label:\n\nTrading off memory and model 
performance with LoRA\n--------------------------------------------------\n\nIn the preceding example, we ran LoRA 
on two devices. But given LoRA\'s low memory footprint, we can run fine-tuning\non a single device using most 
commodity GPUs which support `bfloat16 <https://\nMetadata: {\'document_id\': \'num-3\'}\n',
            type='text'
        ),
        TextContentItem(
            text='Result 2\nContent: ,\n    and (b) the memory constraints of your hardware.\n\nThe preceding 
command will run a LoRA finetune with torchtune\'s factory settings, but we may want to experiment a bit.\nLet\'s 
take a closer look at some of the :code:`lora_finetune_distributed` config.\n\n.. code-block:: yaml\n\n  # Model 
Arguments\n  model:\n    _component_: lora_llama2_7b\n    lora_attn_modules: [\'q_proj\', \'v_proj\']\n    
lora_rank: 8\n    lora_alpha: 16\n  ...\n\nWe see that the default is to apply LoRA to Q and V projections with a 
rank of 8.\nSome experiments with LoRA have found that it can be beneficial to apply LoRA to all linear layers 
in\nthe self-attention, and to increase the rank to 16 or 32. Note that this is likely to increase our max 
memory,\nbut as long as we keep :code:`rank<<embed_dim`, the impact should be relatively minor.\n\nLet\'s run this 
experiment. We can also increase alpha (in general it is good practice to scale alpha and rank together).\n\n.. 
code-block:: bash\n\n    tune run --nnodes 1 --nproc_per_node 2 lora_finetune_distributed --config llama2/7B_lora 
\\\n    lora_attn_modules=[\'q_proj\',\'k_proj\',\'v_proj\',\'output_proj\'] \\\n    lora_rank=32 lora_alpha=64 
output_dir=./lora_experiment_1\n\nA comparison of the (smoothed) loss curves between this run and our baseline over
the first 500 steps can be seen below.\n\n.. image:: /_static/img/lora_experiment_loss_curves.png\n\n.. not

In [10]:
# we can also query the vector db directly
db_response = client.vector_io.query(
    vector_db_id=vector_db_id,
    query=prompt,
)
rich.print(db_response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/vector-io/query "HTTP/1.1 200 OK"


QueryChunksResponse(
    chunks=[
        Chunk(
            content=',\n    and (b) the memory constraints of your hardware.\n\nThe preceding command will run a 
LoRA finetune with torchtune\'s factory settings, but we may want to experiment a bit.\nLet\'s take a closer look 
at some of the :code:`lora_finetune_distributed` config.\n\n.. code-block:: yaml\n\n  # Model Arguments\n  model:\n
_component_: lora_llama2_7b\n    lora_attn_modules: [\'q_proj\', \'v_proj\']\n    lora_rank: 8\n    lora_alpha: 
16\n  ...\n\nWe see that the default is to apply LoRA to Q and V projections with a rank of 8.\nSome experiments 
with LoRA have found that it can be beneficial to apply LoRA to all linear layers in\nthe self-attention, and to 
increase the rank to 16 or 32. Note that this is likely to increase our max memory,\nbut as long as we keep 
:code:`rank<<embed_dim`, the impact should be relatively minor.\n\nLet\'s run this experiment. We can also increase
alpha (in general it is good practice to scale alpha and rank together).\n\n.. code-block:: bash\n\n    tune run 
--nnodes 1 --nproc_per_node 2 lora_finetune_distributed --config llama2/7B_lora \\\n    
lora_attn_modules=[\'q_proj\',\'k_proj\',\'v_proj\',\'output_proj\'] \\\n    lora_rank=32 lora_alpha=64 
output_dir=./lora_experiment_1\n\nA comparison of the (smoothed) loss curves between this run and our baseline over
the first 500 steps can be seen below.\n\n.. image:: /_static/img/lora_experiment_loss_curves.png\n\n.. note::\n   
The above figure was generated with W&B. You can use torchtune\'s 
:class:`~torchtune.training.metric_logging.WandBLogger`\n    to generate similar loss curves, but you will need to 
install W&B and setup an account separately. For more details on\n    using W&B in torchtune, see our 
":ref:`wandb_logging`" recipe.\n\n.. _lora_tutorial_memory_tradeoff_label:\n\nTrading off memory and model 
performance with LoRA\n--------------------------------------------------\n\nIn the preceding example, we ran LoRA 
on two devices. But given LoRA\'s low memory footprint, we can run fine-tuning\non a single device using most 
commodity GPUs which support `bfloat16 <https://',
            metadata={'document_id': 'num-3', 'token_count': 512.0, 'metadata_token_count': 1.0},
            embedding=None
        ),
        Chunk(
            content=',\n    and (b) the memory constraints of your hardware.\n\nThe preceding command will run a 
LoRA finetune with torchtune\'s factory settings, but we may want to experiment a bit.\nLet\'s take a closer look 
at some of the :code:`lora_finetune_distributed` config.\n\n.. code-block:: yaml\n\n  # Model Arguments\n  model:\n
_component_: lora_llama2_7b\n    lora_attn_modules: [\'q_proj\', \'v_proj\']\n    lora_rank: 8\n    lora_alpha: 
16\n  ...\n\nWe see that the default is to apply LoRA to Q and V projections with a rank of 8.\nSome experiments 
with LoRA have found that it can be beneficial to apply LoRA to all linear layers in\nthe self-attention, and to 
increase the rank to 16 or 32. Note that this is likely to increase our max memory,\nbut as long as we keep 
:code:`rank<<embed_dim`, the impact should be relatively minor.\n\nLet\'s run this experiment. We can also increase
alpha (in general it is good practice to scale alpha and rank together).\n\n.. code-block:: bash\n\n    tune run 
--nnodes 1 --nproc_per_node 2 lora_finetune_distributed --config llama2/7B_lora \\\n    
lora_attn_modules=[\'q_proj\',\'k_proj\',\'v_proj\',\'output_proj\'] \\\n    lora_rank=32 lora_alpha=64 
output_dir=./lora_experiment_1\n\nA comparison of the (smoothed) loss curves between this run and our baseline over
the first 500 steps can be seen below.\n\n.. image:: /_static/img/lora_experiment_loss_curves.png\n\n.. note::\n   
The above figure was generated with W&B. You can use torchtune\'s 
:class:`~torchtune.training.metric_logging.WandBLogger`\n    to generate similar loss curves, but you will need to 
install W&B and setup an account separately

In [11]:
# prompt_context = tool_response.content
prompt_context = "\n".join([c.content for c in db_response.chunks])

In [12]:
messages = [{"role": "system", "content": "You are a helpful assistant."}]
extended_prompt = f"""
Please answer the given query using the context below.

QUERY:
{prompt}

CONTEXT:
{prompt_context}
"""
messages.append({"role": "user", "content": extended_prompt})
rich.print(messages)

[
    {'role': 'system', 'content': 'You are a helpful assistant.'},
    {
        'role': 'user',
        'content': '\nPlease answer the given query using the context below.\n\nQUERY:\nWhat are the top 5 topics 
that were explained? Only list succinct bullet points.\n\nCONTEXT:\n,\n    and (b) the memory constraints of your 
hardware.\n\nThe preceding command will run a LoRA finetune with torchtune\'s factory settings, but we may want to 
experiment a bit.\nLet\'s take a closer look at some of the :code:`lora_finetune_distributed` config.\n\n.. 
code-block:: yaml\n\n  # Model Arguments\n  model:\n    _component_: lora_llama2_7b\n    lora_attn_modules: 
[\'q_proj\', \'v_proj\']\n    lora_rank: 8\n    lora_alpha: 16\n  ...\n\nWe see that the default is to apply LoRA 
to Q and V projections with a rank of 8.\nSome experiments with LoRA have found that it can be beneficial to apply 
LoRA to all linear layers in\nthe self-attention, and to increase the rank to 16 or 32. Note that this is likely to
increase our max memory,\nbut as long as we keep :code:`rank<<embed_dim`, the impact should be relatively 
minor.\n\nLet\'s run this experiment. We can also increase alpha (in general it is good practice to scale alpha and
rank together).\n\n.. code-block:: bash\n\n    tune run --nnodes 1 --nproc_per_node 2 lora_finetune_distributed 
--config llama2/7B_lora \\\n    lora_attn_modules=[\'q_proj\',\'k_proj\',\'v_proj\',\'output_proj\'] \\\n    
lora_rank=32 lora_alpha=64 output_dir=./lora_experiment_1\n\nA comparison of the (smoothed) loss curves between 
this run and our baseline over the first 500 steps can be seen below.\n\n.. image:: 
/_static/img/lora_experiment_loss_curves.png\n\n.. note::\n    The above figure was generated with W&B. You can use
torchtune\'s :class:`~torchtune.training.metric_logging.WandBLogger`\n    to generate similar loss curves, but you 
will need to install W&B and setup an account separately. For more details on\n    using W&B in torchtune, see our 
":ref:`wandb_logging`" recipe.\n\n.. _lora_tutorial_memory_tradeoff_label:\n\nTrading off memory and model 
performance with LoRA\n--------------------------------------------------\n\nIn the preceding example, we ran LoRA 
on two devices. But given LoRA\'s low memory footprint, we can run fine-tuning\non a single device using most 
commodity GPUs which support `bfloat16 <https://\n,\n    and (b) the memory constraints of your hardware.\n\nThe 
preceding command will run a LoRA finetune with torchtune\'s factory settings, but we may want to experiment a 
bit.\nLet\'s take a closer look at some of the :code:`lora_finetune_distributed` config.\n\n.. code-block:: 
yaml\n\n  # Model Arguments\n  model:\n    _component_: lora_llama2_7b\n    lora_attn_modules: [\'q_proj\', 
\'v_proj\']\n    lora_rank: 8\n    lora_alpha: 16\n  ...\n\nWe see that the default is to apply LoRA to Q and V 
projections with a rank of 8.\nSome experiments with LoRA have found that it can be beneficial to apply LoRA to all
linear layers in\nthe self-attention, and to increase the rank to 16 or 32. Note that this is likely to increase 
our max memory,\nbut as long as we keep :code:`rank<<embed_dim`, the impact should be relatively minor.\n\nLet\'s 
run this experiment. We can also increase alpha (in general it is good practice to scale alpha and rank 
together).\n\n.. code-block:: bash\n\n    tune run --nnodes 1 --nproc_per_node 2 lora_finetune_distributed --config
llama2/7B_lora \\\n    lora_attn_modules=[\'q_proj\',\'k_proj\',\'v_proj\',\'output_proj\'] \\\n    lora_rank=32 
lora_alpha=64 output_dir=./lora_experiment_1\n\nA comparison of the (smoothed) loss curves between this run and our
baseline over the first 500 steps can be seen below.\n\n.. image:: 
/_static/img/lora_experiment_loss_curves.png\n\n.. note::\n    The above figure was generated with W&B. You can use
torchtune\'s :class:`~torchtune.training.metric_logging.WandBLogger`\n    to generate similar loss curves, but you 
will need 

In [13]:
response = client.inference.chat_completion(
    messages=messages,
    model_id="llama3.2:3b-instruct-fp16",
    timeout=600
)
rich.print(response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/inference/chat-completion "HTTP/1.1 200 OK"


ChatCompletionResponse(
    completion_message=CompletionMessage(
        content='Here are the top 5 topics that were explained:\n\n* LoRA (Low-Rank Adaptation)\n* Memory 
constraints of hardware\n* Fine-tuning with LoRA on a single device\n* Experimenting with different LoRA 
configurations\n* Trading off memory and model performance with LoRA',
        role='assistant',
        stop_reason='end_of_turn',
        tool_calls=[]
    ),
    logprobs=None,
    metrics=[
        Metric(metric='prompt_tokens', value=1592.0, unit=None),
        Metric(metric='completion_tokens', value=68.0, unit=None),
        Metric(metric='total_tokens', value=1660.0, unit=None)
    ]
)

In [14]:
# Left as an extra exercise for the reader
from llama_stack_client import Agent, AgentEventLogger

rag_agent = Agent(
    client, 
    model="llama3.2:3b-instruct-fp16",
    instructions="You are a helpful assistant",
    tools = [
        {
          "name": "builtin::rag/knowledge_search",
          "args" : {
            "vector_db_ids": [vector_db_id],
          }
        }
    ],
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=builtin%3A%3Arag%2Fknowledge_search "HTTP/1.1 200 OK"


In [15]:
import uuid
from llama_stack_client.lib.agents.event_logger import EventLogger
user_prompts = [
    "What is Lora?"
]
session_id = rag_agent.create_session(f"rag session-{uuid.uuid4()}")
for prompt in user_prompts:
    rich.print(f"User> {prompt}")
    response = rag_agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )
    for log in EventLogger().log(response):
        log.print()

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/326578e4-7e72-4068-bbd5-1804814f17a3/session "HTTP/1.1 200 OK"


User> What is Lora?

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents/326578e4-7e72-4068-bbd5-1804814f17a3/session/f02e6687-c49b-43cc-b489-444f22205294/turn "HTTP/1.1 200 OK"


inference> [knowledge_search(query="Lora")]
tool_execution> Tool:knowledge_search Args:{'query': 'Lora'}
tool_execution> Tool:knowledge_search Response:[TextContentItem(text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n', type='text'), TextContentItem(text="Result 1\nContent: .\n\n*Sounds great! How do I use it?*\n\nYou can finetune using any of our recipes with the ``lora_`` prefix, e.g. :ref:`lora_finetune_single_device<lora_finetune_recipe_label>`. These recipes utilize\nLoRA-enabled model builders, which we support for all our models, and also use the ``lora_`` prefix, e.g.\nthe :func:`torchtune.models.llama3.llama3` model has a corresponding :func:`torchtune.models.llama3.lora_llama3`.\nWe aim to provide a comprehensive set of configurations to allow you to get started with training with LoRA quickly,\njust specify any config with ``_lora`` in its name, e.g:\n\n.. code-block:: bash\n\n  tune run lora_finetune_single_device --config llama3/8B_lor